In [3]:
paths = [   "output/netcdfs/landsat8_nbar/water_classify.nc",
            "output/netcdfs/sentinel_2_ab/water_classify.nc",
            "output/netcdfs/sentinel_1_gamma/water_classify.nc"]

In [4]:
import xarray as xr  
s1 = xr.open_dataset(paths[2])
s2 = xr.open_dataset(paths[1])
ls8 = xr.open_dataset(paths[0])

In [7]:
from functools import reduce
merged = reduce(lambda ds_a, ds_b: ds_a.merge(ds_b), [s1,s2,ls8])

### Create Movie function

In [8]:
from IPython.display import HTML, display, Image
from tempfile import NamedTemporaryFile
import matplotlib.pyplot as plt
plt.rcParams["animation.html"] = "jshtml"
import matplotlib.animation as animation
from skimage.transform import resize as imresize
%matplotlib inline
import numpy as np

def create_movie(frames:np.array,
                 text:str=None,
                 fps:int=5,
                 width:int=10,
                 height:int=5,
                 align:bool=True):
    
    
    if text is None:
        text = ['' for s in range(len(frames))]
    ref_shape = frames[0].shape
    fig, ax = plt.subplots()
    ax.set_aspect('equal')
    ax.axis('off')
    
    im = ax.imshow(frames[0])
    _text = ax.text(20, 100, text[0], fontsize=14, color='white')
    im.set_clim([0,1])
    fig.set_size_inches([width,height])
    plt.tight_layout()
    
    def update_img(n):
        frame = imresize(frames[n], ref_shape, order=3)
        _text.set_text(text[n])
        im.set_data(frame)
        return im,
    
    def init():
        im.set_data(frames[0])
        return im,
       
    anim = animation.FuncAnimation(fig, update_img, len(frames), init_func=init, interval=1000);
        
    plt.close(anim._fig)
    return display(HTML(anim.to_jshtml()));

### Application

In [9]:
create_movie(merged.wofs)

/usr/local/lib/python3.6/dist-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/usr/local/lib/python3.6/dist-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "
